In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization, AveragePooling2D
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

### VGG 모델 구현하기

In [ ]:
# MODEL
vgg = Sequential()

# CONV1
vgg.add(Conv2D(filters = 64, kernel_size = 3, padding = "same", input_shape = (224,224,3), activation = "relu"))
vgg.add(Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(MaxPooling2D(2))

# CONV2
vgg.add(Conv2D(filters = 128, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(Conv2D(filters = 128, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(MaxPooling2D(2))

# CONV3
vgg.add(Conv2D(filters = 256, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(Conv2D(filters = 256, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(Conv2D(filters = 256, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(MaxPooling2D(2))

# CONV4
vgg.add(Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(MaxPooling2D(2))

# CONV5
vgg.add(Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu"))
vgg.add(MaxPooling2D(2))

# FC
vgg.add(Flatten())
vgg.add(Dense(4096, activation = "relu"))
vgg.add(Dense(4096, activation = "relu"))
vgg.add(Dense(1000, activation = "softmax"))

In [ ]:
vgg.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_55 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_56 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_57 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_58 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 56, 56, 128)      0         
 g2D)                                                 

In [ ]:
from keras.layers import Concatenate

### 인셉션 이해하기

In [ ]:
def inception(x, filters_1x1, filters_3x3_reduce,filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj) :
  conv_1x1 = Conv2D(filters_1x1, padding = "same", kernel_size = 1, name='conv_1x1')(x)

  conv_3x3_pre = Conv2D(filters_3x3_reduce, kernel_size = 1, padding = "same")(x)
  conv_3x3 = Conv2D(filters_3x3, kernel_size=3, padding = "same")(conv_3x3_pre)

  conv_5x5_pre = Conv2D(filters_5x5_reduce, kernel_size = 1, padding = "same")(x)
  conv_5x5 = Conv2D(filters_5x5, kernel_size = 5, padding = "same")(conv_5x5_pre)

  max_pooling = MaxPooling2D(3, strides=1, padding = "same")(x)
  max_1x1 = Conv2D(filters_pool_proj, kernel_size = 1, padding = "same")(max_pooling) 

  return Concatenate(axis = -1)([conv_1x1, conv_3x3, conv_5x5, max_1x1])

In [ ]:
from keras import Input

In [ ]:
input_layer = Input(shape=(224,224,3))

In [ ]:
y = inception(input_layer, 64, 96, 128, 16, 32, 32)

In [ ]:
y.shape

TensorShape([None, 224, 224, 256])

### 경량 클래스 사용 해보기

In [ ]:
def inception(self, x, filters_1x1, filters_3x3_reduce,filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj) :
  self.x = x
  self.filters_1x1 = filters_1x1
  self.filters_3x3_reduce = filters_3x3_reduce
  self.filters_3x3 = filters_3x3
  self.filters_5x5_reduce = filters_5x5_reduce
  self.filters_5x5 = filters_5x5
  self.filters_pool_proj = filters_pool_proj

  conv_1x1 = Conv2D(self.filters_1x1, padding = "same", kernel_size = 1, name='conv_1x1', activation = "relu")(self.x)

  conv_3x3_pre = Conv2D(self.filters_3x3_reduce, kernel_size = 1, padding = "same", activation = "relu")(self.x)
  conv_3x3 = Conv2D(self.filters_3x3, kernel_size=3, padding = "same", activation = "relu")(conv_3x3_pre)

  conv_5x5_pre = Conv2D(self.filters_5x5_reduce, kernel_size = 1, padding = "same", activation = "relu")(self.x)
  conv_5x5 = Conv2D(self.filters_5x5, kernel_size = 5, padding = "same", activation = "relu")(conv_5x5_pre)

  max_pooling = MaxPooling2D(3, strides=1, padding = "same")(self.x)
  max_1x1 = Conv2D(self.filters_pool_proj, kernel_size = 1, padding = "same")(max_pooling) 

  return Concatenate(axis = 3)([conv_1x1, conv_3x3, conv_5x5, max_1x1])

In [ ]:
Inception = type("stats", (object,), {"func" : inception})

In [ ]:
inception_module = Inception()

In [ ]:
inception_module.func(input_layer, 64, 96, 128, 16, 32, 32)

<KerasTensor: shape=(None, 224, 224, 256) dtype=float32 (created by layer 'concatenate_7')>

In [ ]:
from keras.layers import Conv2D, BatchNormalization, Activation, Add

### ResNet 구현하기

In [ ]:
def bottleneck_residual_block(x, filter, reduce = False, s = 2) :
  f1, f2, f3 = filter 
  
  x_self = x


  if reduce : # 지름길 가는 경우, 지름길 값과 1번째 층 생성
    x_self = Conv2D(filters = f3, kernel_size = 1, strides = s, padding = "same")(x_self)
    x_self = BatchNormalization()(x_self)
    
    conv_1x1_pre = Conv2D(filters = f1, kernel_size = 1, padding = "same")(x)
    conv_1x1_pre = BatchNormalization()(conv_1x1_pre)
    conv_1x1_pre = Activation("relu")(conv_1x1_pre)

  else : # 지름길을 가지 않는 경우 = 1번째 층 생성
    conv_1x1_pre = Conv2D(filters = f1, kernel_size = 1, name = "5", padding = "same")(x_self)
    conv_1x1_pre = BatchNormalization()(conv_1x1_pre)
    conv_1x1_pre = Activation("relu")(conv_1x1_pre)

  # 공통 작업 나머지 층 생성
  conv_3x3 = Conv2D(filters = f2, kernel_size = 3, name = "3", padding = "same")(conv_1x1_pre)
  conv_3x3 = BatchNormalization()(conv_3x3)
  conv_3x3 = Activation("relu")(conv_3x3)

  conv_1x1_post = Conv2D(filters = f3, kernel_size = 1, name = "4", padding = "same")(conv_3x3)
  conv_1x1_post = BatchNormalization()(conv_1x1_post)
  
  # 224, 224, 3 != 224,224,256
  if reduce :
    output = Add()([x_self, conv_1x1_post])
    output = Activation("relu")(output)
  else :
    output = Activation("relu")(conv_1x1_post)

  return output

In [ ]:
input_layer = Input(shape=(224,224,3))

In [ ]:
filter = (64,64,256)

In [ ]:
bottleneck_residual_block(input_layer, filter)

<KerasTensor: shape=(None, 224, 224, 256) dtype=float32 (created by layer 'activation_56')>

In [ ]:
from keras import Model

In [ ]:
def ResNet50(input_shape, classes) :
  input_layer = Input(input_shape)
  print(input_layer)
  #1
  step_1 = Conv2D(filters = 64, kernel_size = 7, strides = 2, padding = "same")(input_layer)
  step_1 = BatchNormalization(axis = 3)(step_1)
  step_1 = Activation("relu")(step_1)
  step_1 = MaxPooling2D(3, strides = 2, padding = "same")(step_1)

  #2
  step_2 = bottleneck_residual_block(x, (64,64,256), reduce = True, s = 1)
  step_2 = bottleneck_residual_block(x, (64,64,256))
  step_2 = bottleneck_residual_block(x, (64,64,256))

  #3
  step_3 = bottleneck_residual_block(x, (128,128,256), reduce = True, s = 2)
  step_3 = bottleneck_residual_block(x, (128,128,256))
  step_3 = bottleneck_residual_block(x, (128,128,256))
  step_3 = bottleneck_residual_block(x, (128,128,256))

  #4
  step_4 = bottleneck_residual_block(x, (256,256,1024))
  step_4 = bottleneck_residual_block(x, (256,256,1024))
  step_4 = bottleneck_residual_block(x, (256,256,1024))
  step_4 = bottleneck_residual_block(x, (256,256,1024))
  step_4 = bottleneck_residual_block(x, (256,256,1024))
  step_4 = bottleneck_residual_block(x, (256,256,1024))

  #5
  step_5 = bottleneck_residual_block(x, (512,512,2048))
  step_5 = bottleneck_residual_block(x, (512,512,2048))
  step_5 = bottleneck_residual_block(x, (512,512,2048))

  # Pooling
  pooling = AveragePooling2D(1)(step_5)
  
  # print
  flat = Flatten()(pooling)
  dens = Dense(classes, activation = "softmax")(flat)

  model = Model(inputs = input_layer, outputs = dens)

  return model

In [ ]:
ResNet50((224,224,3), 1000)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_25'), name='input_25', description="created by layer 'input_25'")


ValueError: ignored

In [ ]:
x = Conv2D(filters = 64, kernel_size = 7, strides = 2, padding = "same")(input_layer)
x = BatchNormalization(axis = 3)(x)
x = Activation("relu")(x)
x = MaxPooling2D(3, strides = 2, padding = "same")(x)



(None, 112, 112, 64)
(None, 56, 56, 64)
